IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M5\NO_FAULT5


In [3]:
guasto=0

VOLO m5 NO FAULT

In [4]:
rcou_m5_nofault = pd.read_csv("RCOU.csv")
rcou_m5_nofault = rcou_m5_nofault.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m5_nofault = rcou_m5_nofault[((rcou_m5_nofault['C9'] >= potenza) & (rcou_m5_nofault['C10'] >= potenza) & (rcou_m5_nofault['C11'] >= potenza) & (rcou_m5_nofault['C12'] >= potenza) & (rcou_m5_nofault['C13'] >= potenza) & (rcou_m5_nofault['C14']>= potenza))]
rcou_m5_nofault=rcou_m5_nofault.reset_index(drop=True)
display(rcou_m5_nofault)

,TimeUS,C9,C10,C11,C12,C13,C14
0,102505087,1385,1404,1405,1384,1392,1397
1,102547919,1440,1396,1417,1420,1435,1401
2,102558488,1407,1441,1429,1419,1409,1439
3,102568605,1458,1401,1427,1434,1452,1407
4,102578804,1501,1362,1438,1434,1501,1362
...,...,...,...,...,...,...,...
4949,162697251,1552,1412,1557,1405,1390,1569
4950,162729893,1554,1397,1557,1393,1403,1550
4951,162753701,1563,1375,1542,1403,1404,1541
4952,162764057,1547,1391,1519,1426,1357,1572


In [5]:
min=rcou_m5_nofault['TimeUS'][0]
max=rcou_m5_nofault['TimeUS'][len(rcou_m5_nofault)-1]
print(max)
print(min)

162774208
102505087


In [6]:
xkf1_m5_nofault = pd.read_csv("XKF1_0.csv")
xkf1_m5_nofault = xkf1_m5_nofault.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m5_nofault = xkf1_m5_nofault[((xkf1_m5_nofault['TimeUS'] >= min) & (xkf1_m5_nofault['TimeUS'] <= max	))]
xkf1_m5_nofault = xkf1_m5_nofault.reset_index(drop=True)
print(xkf1_m5_nofault)

         TimeUS  Roll  Pitch    Yaw
0     102513035  1.85  -1.49  10.85
1     102523905  1.94  -1.53  10.85
2     102535459  1.99  -1.53  10.84
3     102545883  1.98  -1.51  10.84
4     102555882  1.93  -1.48  10.85
...         ...   ...    ...    ...
5479  162727781 -2.37  -0.06   6.21
5480  162741639 -2.40  -0.08   6.17
5481  162751479 -2.43  -0.06   6.12
5482  162761552 -2.44  -0.01   6.10
5483  162772111 -2.53  -0.03   6.05

[5484 rows x 4 columns]


In [7]:
att_m5_nofault = pd.read_csv("ATT.csv")
att_m5_nofault = att_m5_nofault.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m5_nofault = att_m5_nofault[((att_m5_nofault['TimeUS'] >= min) & (att_m5_nofault['TimeUS'] <= max	))]
att_m5_nofault=att_m5_nofault.reset_index(drop=True)
print(att_m5_nofault)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     102515089    -0.02  1.85      0.14  -1.49   10.80  10.85
1     102525154    -0.04  1.94      0.15  -1.53   10.80  10.85
2     102536919    -0.06  1.99      0.15  -1.53   10.80  10.84
3     102547199    -0.09  1.98      0.16  -1.51   10.80  10.84
4     102557820    -0.10  1.93      0.17  -1.48   10.80  10.85
...         ...      ...   ...       ...    ...     ...    ...
5479  162729207     0.38 -2.37     -1.24  -0.06    2.66   6.21
5480  162742824     0.42 -2.40     -1.26  -0.08    2.66   6.17
5481  162752885     0.47 -2.43     -1.29  -0.06    2.66   6.12
5482  162763219     0.52 -2.44     -1.32  -0.01    2.66   6.10
5483  162773559     0.58 -2.53     -1.35  -0.03    2.66   6.05

[5484 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m5_nofault)>len(xkf1_m5_nofault):
    to_add=att_m5_nofault[len(xkf1_m5_nofault):]
    att_m5_nofault=att_m5_nofault[:len(xkf1_m5_nofault)]

if len(xkf1_m5_nofault)>len(att_m5_nofault):
    to_add=xkf1_m5_nofault[len(att_m5_nofault):]

for idx,i in enumerate(att_m5_nofault['Roll']):
    if(xkf1_m5_nofault['Roll'][idx] != i):
        att_m5_nofault['Roll'][idx] = ((att_m5_nofault['Roll'][idx] + xkf1_m5_nofault['Roll'][idx])/2)

for idx,i in enumerate(att_m5_nofault['Pitch']):
    if(xkf1_m5_nofault['Pitch'][idx] != i):
        att_m5_nofault['Pitch'][idx] = ((att_m5_nofault['Pitch'][idx] + xkf1_m5_nofault['Pitch'][idx])/2)

for idx,i in enumerate(att_m5_nofault['Yaw']):
    if(xkf1_m5_nofault['Yaw'][idx] != i):
        att_m5_nofault['Yaw'][idx] = ((att_m5_nofault['Yaw'][idx] + xkf1_m5_nofault['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m5_nofault=pd.concat([att_m5_nofault,to_add])

print(att_m5_nofault)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     102515089    -0.02  1.85      0.14  -1.49   10.80  10.85
1     102525154    -0.04  1.94      0.15  -1.53   10.80  10.85
2     102536919    -0.06  1.99      0.15  -1.53   10.80  10.84
3     102547199    -0.09  1.98      0.16  -1.51   10.80  10.84
4     102557820    -0.10  1.93      0.17  -1.48   10.80  10.85
...         ...      ...   ...       ...    ...     ...    ...
5479  162729207     0.38 -2.37     -1.24  -0.06    2.66   6.21
5480  162742824     0.42 -2.40     -1.26  -0.08    2.66   6.17
5481  162752885     0.47 -2.43     -1.29  -0.06    2.66   6.12
5482  162763219     0.52 -2.44     -1.32  -0.01    2.66   6.10
5483  162773559     0.58 -2.53     -1.35  -0.03    2.66   6.05

[5484 rows x 7 columns]


C:\Users\mgale\AppData\Local\Temp\ipykernel_12508\759630520.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m5_nofault['Yaw'][idx] = ((att_m5_nofault['Yaw'][idx] + xkf1_m5_nofault['Yaw'][idx])/2)


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m5_nofault = pd.read_csv("ESC_0.csv")
esc_0_m5_nofault = esc_0_m5_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m5_nofault = esc_0_m5_nofault[((esc_0_m5_nofault['TimeUS'] >= min) & (esc_0_m5_nofault['TimeUS'] <= max))]
esc_0_m5_nofault=esc_0_m5_nofault.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m5_nofault = pd.read_csv("ESC_1.csv")
esc_1_m5_nofault = esc_1_m5_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m5_nofault = esc_1_m5_nofault[((esc_1_m5_nofault['TimeUS'] >= min) & (esc_1_m5_nofault['TimeUS'] <= max))]
esc_1_m5_nofault=esc_1_m5_nofault.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m5_nofault = pd.read_csv("ESC_2.csv")
esc_2_m5_nofault = esc_2_m5_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m5_nofault = esc_2_m5_nofault[((esc_2_m5_nofault['TimeUS'] >= min) & (esc_2_m5_nofault['TimeUS'] <= max))]
esc_2_m5_nofault=esc_2_m5_nofault.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m5_nofault = pd.read_csv("ESC_3.csv")
esc_3_m5_nofault = esc_3_m5_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m5_nofault = esc_3_m5_nofault[((esc_3_m5_nofault['TimeUS'] >= min) & (esc_3_m5_nofault['TimeUS'] <= max))]
esc_3_m5_nofault=esc_3_m5_nofault.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m5_nofault = pd.read_csv("ESC_4.csv")
esc_4_m5_nofault = esc_4_m5_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m5_nofault = esc_4_m5_nofault[((esc_4_m5_nofault['TimeUS'] >= min) & (esc_4_m5_nofault['TimeUS'] <= max))]
esc_4_m5_nofault=esc_4_m5_nofault.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m5_nofault = pd.read_csv("ESC_5.csv")
esc_5_m5_nofault = esc_5_m5_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m5_nofault = esc_5_m5_nofault[((esc_5_m5_nofault['TimeUS'] >= min) & (esc_5_m5_nofault['TimeUS'] <= max))]
esc_5_m5_nofault=esc_5_m5_nofault.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m5_nofault = pd.read_csv("IMU_0.csv")
imu_0_m5_nofault = imu_0_m5_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m5_nofault = imu_0_m5_nofault[((imu_0_m5_nofault['TimeUS'] >= min) & (imu_0_m5_nofault['TimeUS'] <= max))]

imu_1_m5_nofault = pd.read_csv("IMU_1.csv")
imu_1_m5_nofault = imu_1_m5_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m5_nofault = imu_1_m5_nofault[((imu_1_m5_nofault['TimeUS'] >= min) & (imu_1_m5_nofault['TimeUS'] <= max))]

imu_2_m5_nofault = pd.read_csv("IMU_2.csv")
imu_2_m5_nofault = imu_2_m5_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m5_nofault = imu_2_m5_nofault[((imu_2_m5_nofault['TimeUS'] >= min) & (imu_2_m5_nofault['TimeUS'] <= max))]

imu_m5_nofault = pd.concat((imu_0_m5_nofault, imu_1_m5_nofault, imu_2_m5_nofault))
imu_m5_nofault=imu_m5_nofault.groupby(imu_m5_nofault.TimeUS, as_index=False).mean()

display(imu_m5_nofault)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,102506599,-0.364870,0.051198,0.015439,-0.196391,-0.273725,-9.656544
1,102509578,-0.199651,-0.022153,0.011422,-0.104806,0.072470,-9.512378
2,102511769,0.084841,-0.071096,-0.003225,-0.121335,0.205762,-9.454771
3,102514770,0.295913,-0.067083,-0.007073,-0.237170,0.049783,-9.474679
4,102517655,-0.020380,0.062274,0.011793,-0.314133,0.005627,-9.648276
...,...,...,...,...,...,...,...
20824,162763113,-0.033571,0.084760,-0.042827,0.069566,0.555606,-9.501016
20825,162765819,0.120113,0.012521,-0.068512,0.072377,0.397154,-9.557518
20826,162768625,0.021385,-0.016392,-0.085739,0.055393,0.223394,-9.623663
20827,162770890,-0.150573,-0.009726,-0.084213,0.030201,0.249535,-9.643305


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m5_nofault = pd.merge_ordered(imu_m5_nofault,att_m5_nofault)
m5_nofault=m5_nofault.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m5_nofault = pd.merge_ordered(m5_nofault, esc_0_m5_nofault)
m5_nofault = pd.merge_ordered(m5_nofault, esc_1_m5_nofault)
m5_nofault = pd.merge_ordered(m5_nofault, esc_2_m5_nofault)
m5_nofault = pd.merge_ordered(m5_nofault, esc_3_m5_nofault)
m5_nofault = pd.merge_ordered(m5_nofault, esc_4_m5_nofault)
m5_nofault = pd.merge_ordered(m5_nofault, esc_5_m5_nofault)
m5_nofault=m5_nofault.fillna(method="ffill").fillna(method="bfill")
m5_nofault = pd.merge_ordered(m5_nofault, rcou_m5_nofault)
m5_nofault=m5_nofault.fillna(method="ffill").fillna(method="bfill")

In [12]:
m5_nofault["TimeUS"] = m5_nofault["TimeUS"] - m5_nofault.iloc[0]["TimeUS"]
m5_nofault["TimeUS"] = m5_nofault["TimeUS"].astype(int)
m5_nofault["Guasto"] = guasto

In [13]:
last_cell = m5_nofault.iloc[-1, m5_nofault.columns.get_loc('TimeUS')]
print(last_cell)

60269121


In [14]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [15]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

         TimeUS
0             0
1          2857
2          5714
3          8571
4         11428
...         ...
21091  60256987
21092  60259844
21093  60262701
21094  60265558
21095  60268415

[21096 rows x 1 columns]


In [16]:
m5_nofault_final = pd.merge_ordered(m5_nofault,df)
m5_nofault_final=m5_nofault_final.fillna(method="ffill").fillna(method="bfill")
print(m5_nofault_final)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY      AccZ  \
0             0 -0.364870  0.051198  0.015439 -0.196391 -0.273725 -9.656544   
1          1512 -0.364870  0.051198  0.015439 -0.196391 -0.273725 -9.656544   
2          2857 -0.364870  0.051198  0.015439 -0.196391 -0.273725 -9.656544   
3          4491 -0.199651 -0.022153  0.011422 -0.104806  0.072470 -9.512378   
4          5714 -0.199651 -0.022153  0.011422 -0.104806  0.072470 -9.512378   
...         ...       ...       ...       ...       ...       ...       ...   
55421  60265803 -0.150573 -0.009726 -0.084213  0.030201  0.249535 -9.643305   
55422  60268204 -0.152369  0.006015 -0.076671  0.028353  0.414205 -9.604354   
55423  60268415 -0.152369  0.006015 -0.076671  0.028353  0.414205 -9.604354   
55424  60268472 -0.152369  0.006015 -0.076671  0.028353  0.414205 -9.604354   
55425  60269121 -0.152369  0.006015 -0.076671  0.028353  0.414205 -9.604354   

       DesRoll  Roll  DesPitch  ...  Curr(ESC_4)  R

In [17]:
m5_nofault_final = m5_nofault_final[m5_nofault_final.TimeUS.isin(to_be)]

In [18]:
m5_nofault_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,0,-0.364870,0.051198,0.015439,-0.196391,-0.273725,-9.656544,-0.02,1.85,0.14,...,2.53,2798.0,1.95,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
2,2857,-0.364870,0.051198,0.015439,-0.196391,-0.273725,-9.656544,-0.02,1.85,0.14,...,2.53,2798.0,1.95,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
4,5714,-0.199651,-0.022153,0.011422,-0.104806,0.072470,-9.512378,-0.02,1.85,0.14,...,2.53,2798.0,1.95,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
6,8571,0.084841,-0.071096,-0.003225,-0.121335,0.205762,-9.454771,-0.02,1.85,0.14,...,2.53,2798.0,1.95,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
9,11428,0.295913,-0.067083,-0.007073,-0.237170,0.049783,-9.474679,-0.02,1.85,0.14,...,2.53,2798.0,1.95,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55411,60256987,-0.160556,0.065735,-0.040423,-0.010292,0.480986,-9.432996,0.47,-2.43,-1.29,...,1.41,5723.0,3.25,1563.0,1375.0,1542.0,1403.0,1404.0,1541.0,0.0
55415,60259844,-0.033571,0.084760,-0.042827,0.069566,0.555606,-9.501016,0.52,-2.44,-1.32,...,1.41,5723.0,3.25,1547.0,1391.0,1519.0,1426.0,1357.0,1572.0,0.0
55418,60262701,0.120113,0.012521,-0.068512,0.072377,0.397154,-9.557518,0.52,-2.44,-1.32,...,1.41,5807.0,3.38,1547.0,1391.0,1519.0,1426.0,1357.0,1572.0,0.0
55420,60265558,0.021385,-0.016392,-0.085739,0.055393,0.223394,-9.623663,0.52,-2.44,-1.32,...,1.41,5807.0,3.38,1547.0,1391.0,1519.0,1426.0,1357.0,1572.0,0.0


In [19]:
from datetime import timedelta
m5_nofault_final=m5_nofault_final.reset_index(drop=True)
m5_nofault_final['TimeUS'] = pd.to_datetime(m5_nofault_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m5_nofault_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.364870,0.051198,0.015439,-0.196391,-0.273725,-9.656544,-0.02,1.85,0.14,...,2.53,2798.0,1.95,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
1,00:00:00.002857,-0.364870,0.051198,0.015439,-0.196391,-0.273725,-9.656544,-0.02,1.85,0.14,...,2.53,2798.0,1.95,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
2,00:00:00.005714,-0.199651,-0.022153,0.011422,-0.104806,0.072470,-9.512378,-0.02,1.85,0.14,...,2.53,2798.0,1.95,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
3,00:00:00.008571,0.084841,-0.071096,-0.003225,-0.121335,0.205762,-9.454771,-0.02,1.85,0.14,...,2.53,2798.0,1.95,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
4,00:00:00.011428,0.295913,-0.067083,-0.007073,-0.237170,0.049783,-9.474679,-0.02,1.85,0.14,...,2.53,2798.0,1.95,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21091,00:01:00.256987,-0.160556,0.065735,-0.040423,-0.010292,0.480986,-9.432996,0.47,-2.43,-1.29,...,1.41,5723.0,3.25,1563.0,1375.0,1542.0,1403.0,1404.0,1541.0,0.0
21092,00:01:00.259844,-0.033571,0.084760,-0.042827,0.069566,0.555606,-9.501016,0.52,-2.44,-1.32,...,1.41,5723.0,3.25,1547.0,1391.0,1519.0,1426.0,1357.0,1572.0,0.0
21093,00:01:00.262701,0.120113,0.012521,-0.068512,0.072377,0.397154,-9.557518,0.52,-2.44,-1.32,...,1.41,5807.0,3.38,1547.0,1391.0,1519.0,1426.0,1357.0,1572.0,0.0
21094,00:01:00.265558,0.021385,-0.016392,-0.085739,0.055393,0.223394,-9.623663,0.52,-2.44,-1.32,...,1.41,5807.0,3.38,1547.0,1391.0,1519.0,1426.0,1357.0,1572.0,0.0


## Analisi nel tempo e in frequenza

In [21]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [22]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m5_nofault_final)/350)):
        V1=m5_nofault_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab

In [23]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_12508\1142334899.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_12508\1142334899.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.062372,0.011086,0.728895,0.122247,0.0,91.0,11.309668,3.303893,0.026686,...,8320.608225,2.940256,0.239729,0.538357,2.980630,114.0,111.0,25.434346,8.606460,0
1,1.0,-0.062231,0.011061,0.732446,0.122075,0.0,102.0,11.451014,3.247529,0.026223,...,8006.676800,2.939830,0.239111,0.546171,2.980106,113.0,110.0,25.210311,8.240892,0
2,2.0,-0.062510,0.011057,0.731551,0.122202,0.0,111.0,11.443601,3.035146,0.026083,...,7666.622539,2.939405,0.238495,0.553995,2.979584,112.0,109.0,24.952741,7.785895,0
3,3.0,-0.062547,0.011026,0.742453,0.122095,0.0,76.0,11.520021,3.060060,0.025824,...,7300.796027,2.938983,0.237882,0.561829,2.979066,113.0,110.0,24.661963,7.241657,0
4,4.0,-0.062344,0.011010,0.739970,0.121924,0.0,103.0,11.680437,2.843586,0.025251,...,6909.891450,2.938563,0.237273,0.569674,2.978550,110.0,107.0,24.338363,6.609797,0
5,5.0,-0.062369,0.010979,0.750795,0.121811,0.0,100.0,11.758881,2.699322,0.024816,...,6494.952177,2.938146,0.236666,0.577529,2.978037,113.0,110.0,23.982388,5.893485,0
6,6.0,-0.062621,0.010971,0.751808,0.121907,0.0,73.0,11.755249,3.417518,0.024563,...,6057.368950,2.937731,0.236063,0.585394,2.977527,110.0,109.0,23.594543,5.299169,0
7,7.0,-0.062669,0.010941,0.762613,0.121810,0.0,77.0,11.822706,2.968557,0.024177,...,5944.232105,2.937318,0.235463,0.593269,2.977019,113.0,112.0,23.175382,5.534205,0
8,8.0,-0.062542,0.010916,0.766852,0.121644,0.0,75.0,11.952071,2.443661,0.023642,...,6512.063181,2.936908,0.234866,0.601154,2.976515,112.0,108.0,22.725511,5.882120,0
9,9.0,-0.062575,0.010886,0.777666,0.121538,0.0,74.0,12.022786,3.161362,0.023265,...,7032.388682,2.936500,0.234271,0.609049,2.976013,113.0,109.0,22.245580,6.290009,0


In [24]:
path_file = path_file.replace(r"M5\NO_FAULT5", "")
os.chdir(path_file)
df_merged.to_csv('m5_nofault.csv', index=False)
print(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\
